## PSO Fitting Testing

In [3]:
from pulse2percept.models import BiphasicAxonMapModel, AxonMapModel
from pulse2percept.model_selection import ParticleSwarmOptimizer
from pulse2percept.implants import ArgusII
import shapes

import matplotlib.pyplot as plt
import numpy as np

#### Do 12-005 S2

In [4]:
s2 = shapes.subject_params['12-005']
implant, model = shapes.model_from_params(s2)
model.xystep=0.5

# CHANGING relative_weight will be VERY important
estimator = shapes.BiphasicAxonMapEstimator(implant=implant, model=model)
search_params = {
    'a5' : (-10, 10),
    'a6': (-10, 10),
    'rho': (10, 300),
    'axlambda': (10, 500)
}
# search_params = {
#     'rho': (10, 300),
#     'axlambda': (10, 500)
# }
opt = ParticleSwarmOptimizer(estimator, search_params, max_iter=100, swarm_size=40, has_loss_function=True)

data = shapes.load_shapes("../data/shapes.h5", subjects=['12-005'], stim_class='SingleElectrode')
x = data[['amp1', 'freq', 'pdur', 'electrode1']]
y = data['image']


In [5]:
estimator.score(x, y)

rho=200.000000, axlambda=200.000000, a5=1.081200, a6=-0.353380, score=7.165556


7.165556445500585

In [ ]:
opt.fit(x, y)

rho=67.953196, axlambda=270.961354, a5=0.299952, a6=-3.436341, score=4.498067
rho=148.781611, axlambda=356.611679, a5=5.126375, a6=-3.559404, score=591.276848
rho=98.265550, axlambda=197.117046, a5=-1.549681, a6=9.939638, score=77.870495
rho=122.923078, axlambda=237.723255, a5=-4.503130, a6=2.516829, score=4.574243
rho=273.596978, axlambda=471.310577, a5=-7.039471, a6=8.136399, score=4.172788
rho=233.601475, axlambda=52.027440, a5=1.420281, a6=3.509843, score=1975.303523
rho=183.460964, axlambda=159.094904, a5=-0.565810, a6=4.978662, score=284.870172
rho=181.554737, axlambda=239.577876, a5=8.716585, a6=6.478649, score=10770.557766
rho=83.656904, axlambda=33.233052, a5=-6.821534, a6=8.464923, score=5.373624
rho=288.744111, axlambda=310.605236, a5=-0.726664, a6=9.473838, score=8131.852278
rho=62.242725, axlambda=158.692307, a5=-2.252813, a6=-5.722888, score=4.852459
rho=54.115035, axlambda=426.463435, a5=-1.424941, a6=-3.499058, score=4.220000
rho=268.465357, axlambda=130.581442, a5=1.78